# Offer Allocation Problem

Price discrimination is often achieved by having one stated public price but offer- ing individual consumers discounts or coupons which lower the individual consumer’s price. Finding the op- timal mix of discounts or coupons in order to maximize profitability is an optimal-selection problem, naturally formulated as a constrained quadratic binary model.

Assign $m$ offers to $n$ consumers. <br>
Each offer $m$ can only be given to a small number of consumers $c_{j}$, <br>
And each consumer can only receive a handful of offers $c_{i}$. 
Binary variable $a_{i, j}$ is assigned to each offer-consumer pair. 

**Constraint 1:** <br>
Constraint on the number of consumers given an offer : <br>
$\sum_{i}^{n} a_{i, j} \leq c_{j}$


**Constraint 2:** <br>
Constraint on the number of offers given to a consumer: <br>
$\sum_{i}^{n} a_{i, j} \leq c_{i}$


In [4]:
import dimod
import numpy as np

In [20]:
def offer_allocation(m, n, c_consumers, c_offers, values):
    """Offer Allocation Function
    m: number of offers
    n: number of consumers
    c_consumers: total number of offers per consumer
    total_offers: total number of offers
    """

    cqm = dimod.ConstrainedQuadraticModel()

    assignments = {}
    for i in range(n): 
        # loops over number of consumers (n)
        for j in range(m):
            # loops of number of offers (m)
            # each offer-consumer pair is a binmary variable
            pair = dimod.Binary(f"consumer_{i}_offer_{j}")
            assignments[i, j] = pair
    
    # Constraint 2: each consumer gets c_consumers offers
    for i in range(n):
        cqm.add_constraint_from_comparison(
            dimod.quicksum(assignments[i, j] for j in range(m)
            ) <= c_consumers[i]
        )

    # Constraint 1: each offer can only be given a limited number of times
    for j in range(m):
        cqm.add_constraint_from_comparison(
            dimod.quicksum(assignments[i, j] for i in range(n)
            ) <= c_offers[j]
        )
    
    # Objective: Maximize total value of deal mix per customer offered deals
    cqm.set_objective(
        dimod.quicksum(
            -val * assignments[i, j] * assignments[i, k]
            for (i, j, k), val in values.items()
            if j > k
        ),
    )

    return cqm

In [21]:
m = 5  # number of offers
n = 10  # number of consumers
c_consumers = np.random.randint(1, m, n)  # random number of offers per consumer
c_offers = np.random.randint(1, n, m)  # random number of times each offer can be given
values = {(i, j, k): np.random.rand() for i in range(n) for j in range(m) for k in range(m) if j > k}  # random values for each offer-consumer pair


In [22]:
cqm = offer_allocation(m, n, c_consumers, c_offers, values)

## Solve the Prvoblem by Sampling

In [23]:
from dwave.system import LeapHybridCQMSampler
sampler = LeapHybridCQMSampler()

In [24]:
sampleset = sampler.sample_cqm(cqm)
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
print("{} feasible solutions of {}.".format(len(feasible_sampleset), len(sampleset)))

84 feasible solutions of 120.


Defining a utility function, `print_allocation` to print the returned solutions in an intuitive format

In [28]:
print("Feasible Sampleset:{}".format(feasible_sampleset.first))

Feasible Sampleset:Sample(sample={'consumer_0_offer_0': 1.0, 'consumer_0_offer_1': 1.0, 'consumer_0_offer_2': 0.0, 'consumer_0_offer_3': 0.0, 'consumer_0_offer_4': 1.0, 'consumer_1_offer_0': 0.0, 'consumer_1_offer_1': 0.0, 'consumer_1_offer_2': 0.0, 'consumer_1_offer_3': 0.0, 'consumer_1_offer_4': 0.0, 'consumer_2_offer_0': 0.0, 'consumer_2_offer_1': 1.0, 'consumer_2_offer_2': 1.0, 'consumer_2_offer_3': 0.0, 'consumer_2_offer_4': 0.0, 'consumer_3_offer_0': 1.0, 'consumer_3_offer_1': 1.0, 'consumer_3_offer_2': 1.0, 'consumer_3_offer_3': 0.0, 'consumer_3_offer_4': 0.0, 'consumer_4_offer_0': 0.0, 'consumer_4_offer_1': 1.0, 'consumer_4_offer_2': 1.0, 'consumer_4_offer_3': 0.0, 'consumer_4_offer_4': 1.0, 'consumer_5_offer_0': 0.0, 'consumer_5_offer_1': 0.0, 'consumer_5_offer_2': 0.0, 'consumer_5_offer_3': 0.0, 'consumer_5_offer_4': 0.0, 'consumer_6_offer_0': 0.0, 'consumer_6_offer_1': 1.0, 'consumer_6_offer_2': 0.0, 'consumer_6_offer_3': 0.0, 'consumer_6_offer_4': 0.0, 'consumer_7_offer_0':

In [ ]:
def print_diet(sample):
    diet = {food: round(quantity, 1) for food, quantity in sample.items()}
    print(f"Diet: {diet}")
    taste_total = sum(foods[food]['Taste'] * amount for food, amount in sample.items())
    cost_total = sum(foods[food]['Cost'] * amount for food, amount in sample.items())
    print(f"Total Taste of {round(taste_total, 2)} at Cost ${round(cost_total, 2)}")
    for constraint in cqm.iter_constraint_data(sample):
        print(f"{constraint.label} (nominal: {constraint.rhs_energy}) : {round(constraint.lhs_energy)}")